In [3]:
import pandas as pd
import numpy as np
import nltk
import spacy

In [2]:
nlp = spacy.load('en_core_web_sm')

In [6]:
data = pd.read_csv('../data_bin.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,binary_valence
0,0,2949,https://www.last.fm/music/skywave/_/another%2b...,Another Love,Skywave,"['explosive', 'euphoric']",5,5.440000,4.582000,4.548000,3eba75ff-50d1-48eb-b39f-2e1e456c0847,NaN,shoegaze,"[Intro]\nIt's Rittz, bitch\n\n[Verse 1]\nI can...",1
1,1,65958,https://www.last.fm/music/neneh%2bcherry/_/pea...,Peace In Mind,Neneh Cherry,['reflective'],4,4.841667,2.815000,4.950000,05011436-be05-433d-b7d1-405f14b45838,3qij053VLKWDprTlF6REDl,trip-hop,I've got a little piece of mind\nFrom the comf...,0
2,2,43234,https://www.last.fm/music/adam%2bgreen/_/goblin,Goblin,Adam Green,['silly'],1,6.720000,4.860000,6.500000,93f50361-0581-4dd6-a294-93ca5bef904f,122tMrH7PcUWMN4AcklyU0,folk,When she stopped talking to me\nI knew that sh...,1
3,3,5766,https://www.last.fm/music/ceremony/_/i%2bwant%...,I Want To Put This To An End,Ceremony,['intense'],2,3.995000,6.135000,4.585000,3b09dada-b877-4bda-b35a-34a1d28b1461,3HBwKXoQnPdi8Tzmioiwyv,hardcore,"My heart beats in slow songs, pumping moments ...",0
4,4,61410,https://www.last.fm/music/beyond%2bcreation/_/...,Coexistence,Beyond Creation,['technical'],2,5.693333,5.476275,6.390392,e4348ebc-97f2-4bcb-af2e-2998d0fae905,3TfnNa2rOCuGZi4IBWtYVE,technical death metal,When the universal darkness is covering the ea...,1


In [7]:
affectDB = pd.read_csv("affect.csv")
affectDB.head()

,Unnamed: 0,Word,V.Mean.Sum,V.SD.Sum,V.Rat.Sum,A.Mean.Sum,A.SD.Sum,A.Rat.Sum,D.Mean.Sum,D.SD.Sum,...,A.Rat.L,A.Mean.H,A.SD.H,A.Rat.H,D.Mean.L,D.SD.L,D.Rat.L,D.Mean.H,D.SD.H,D.Rat.H
0,1,aardvark,6.26,2.21,19,2.41,1.40,22,4.27,1.75,...,11,2.55,1.29,11,4.12,1.64,8,4.43,1.99,7
1,2,abalone,5.30,1.59,20,2.65,1.90,20,4.95,1.79,...,12,2.38,1.92,8,5.55,2.21,11,4.36,1.03,11
2,3,abandon,2.84,1.54,19,3.73,2.43,22,3.32,2.50,...,11,3.82,2.14,11,2.77,2.09,13,4.11,2.93,9
3,4,abandonment,2.63,1.74,19,4.95,2.64,21,2.64,1.81,...,14,5.29,2.63,7,2.31,1.45,16,3.08,2.19,12
4,5,abbey,5.85,1.69,20,2.20,1.70,20,5.00,2.02,...,9,2.55,1.92,11,4.83,2.18,18,5.43,1.62,7


In [10]:
toDrop = data.index[data['lyrics'] == "None"].tolist()
print(toDrop)
data.drop(toDrop, axis = 0, inplace = True)
#to test if this works
print(data.index[data['lyrics'] == "None"].tolist())

[]
[]


In [69]:
calculatedValence = []
calculatedDominance = []
calculatedArousal = []
avgV = data['valence_tags'].mean()
avgD = data['dominance_tags'].mean()
avgA = data['arousal_tags'].mean()
for i in range(len(data)):
    doc = nlp(data.iloc[i]['lyrics'])
    lemmas = [token.lemma_ for token in doc]
    counter = 0
    valence = 0.0
    dominance = 0.0
    arousal = 0.0
    for lemma in lemmas:
        index = affectDB.loc[affectDB['Word'] == lemma, ['V.Mean.Sum', 'D.Mean.Sum', 'A.Mean.Sum']]
        #print(index)
        if not index.empty:
            valence += index['V.Mean.Sum'].item()
            dominance += index['D.Mean.Sum'].item()
            arousal += index['A.Mean.Sum'].item()
            counter += 1
    if counter > 0:
        calculatedValence.append(valence / counter)
        calculatedDominance.append(dominance / counter)
        calculatedArousal.append(arousal / counter)
    else:
        calculatedValence.append(avgV)
        calculatedDominance.append(avgD)
        calculatedArousal.append(avgA)
#     print(i)
#     print(calculatedValence[i], calculatedDominance[i], calculatedArousal[i], counter)
#     print(calculatedValence)
data['calculatedValence'] = calculatedValence
data['calculatedDominance'] = calculatedDominance
data['calculatedArousal'] = calculatedArousal


            
            


In [63]:
doc = nlp(data.iloc[3]['lyrics'])
lemmas = [token.lemma_ for token in doc]
counter = 0
valence = 0.0
dominance = 0.0
arousal = 0.0
for lemma in lemmas:
    index = affectDB.loc[affectDB['Word'] == lemma, ['V.Mean.Sum', 'D.Mean.Sum', 'A.Mean.Sum']]
    #print(index)
    if not index.empty:
        valence += index['V.Mean.Sum'].item()
        dominance += index['D.Mean.Sum'].item()
        arousal += index['A.Mean.Sum'].item()
        counter += 1
print(valence / counter)
print(dominance / counter)
print(arousal / counter)
print(counter)


5.3822641509433975
5.339811320754718
4.143773584905661
53


In [53]:
print(avgV)

4.29512255735349


In [70]:
data.head(200)

,Unnamed: 0.1,Unnamed: 0,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics,binary_valence,calculatedValence,calculatedDominance,calculatedArousal
0,0,2949,https://www.last.fm/music/skywave/_/another%2b...,Another Love,Skywave,"['explosive', 'euphoric']",5,5.440000,4.582000,4.548000,3eba75ff-50d1-48eb-b39f-2e1e456c0847,NaN,shoegaze,"[Intro]\nIt's Rittz, bitch\n\n[Verse 1]\nI can...",1,5.828876,5.634538,4.164418
1,1,65958,https://www.last.fm/music/neneh%2bcherry/_/pea...,Peace In Mind,Neneh Cherry,['reflective'],4,4.841667,2.815000,4.950000,05011436-be05-433d-b7d1-405f14b45838,3qij053VLKWDprTlF6REDl,trip-hop,I've got a little piece of mind\nFrom the comf...,0,6.093000,5.622000,4.091375
2,2,43234,https://www.last.fm/music/adam%2bgreen/_/goblin,Goblin,Adam Green,['silly'],1,6.720000,4.860000,6.500000,93f50361-0581-4dd6-a294-93ca5bef904f,122tMrH7PcUWMN4AcklyU0,folk,When she stopped talking to me\nI knew that sh...,1,5.738000,5.548000,4.086800
3,3,5766,https://www.last.fm/music/ceremony/_/i%2bwant%...,I Want To Put This To An End,Ceremony,['intense'],2,3.995000,6.135000,4.585000,3b09dada-b877-4bda-b35a-34a1d28b1461,3HBwKXoQnPdi8Tzmioiwyv,hardcore,"My heart beats in slow songs, pumping moments ...",0,5.382264,5.339811,4.143774
4,4,61410,https://www.last.fm/music/beyond%2bcreation/_/...,Coexistence,Beyond Creation,['technical'],2,5.693333,5.476275,6.390392,e4348ebc-97f2-4bcb-af2e-2998d0fae905,3TfnNa2rOCuGZi4IBWtYVE,technical death metal,When the universal darkness is covering the ea...,1,5.810159,5.637222,4.049206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,284,17510,https://www.last.fm/music/stray%2btheories/_/h...,Here I Was,Stray Theories,"['lonely', 'introspective']",4,2.888340,3.530596,3.286936,4d425cf3-c34d-4752-9c98-804c2485dd46,NaN,ambient,"[NORMA, spoken]\nNow go\n\n[JOE, spoken]\nNext...",0,6.028333,5.861000,3.936667
286,286,5705,https://www.last.fm/music/man%2bman/_/push%2bt...,Push The Eagle's Stomach,Man Man,"['intense', 'quirky']",2,6.040000,5.475000,5.695000,00286cef-e7e0-43aa-b9e6-783c99a8d520,5ykVoplqZmme1YvGyg97AF,experimental,I don't wannna know your name (my vice grips)\...,1,5.501584,5.391188,3.951584
288,288,43460,https://www.last.fm/music/king%2bdude/_/spider...,Spiders in Her Hair,King Dude,"['silly', 'negative']",9,2.890278,4.869444,4.091944,f6500372-8746-449f-932c-3a228dd8b44c,3BE1W1vnHMpdUmzLxaYqq6,k-pop,[Verse 1]\nI always thought I was blacker than...,0,5.811047,5.345930,4.255930
289,289,12838,https://www.last.fm/music/mattafix/_/forgotten,Forgotten,Mattafix,['paranoid'],1,2.900000,5.090000,3.810000,60652715-d32e-4b39-85a7-b8cfc6754022,0UpTDnNUditHH0B5kLCRWm,alternative,Vacuum. Not even a court room. Just me in a sm...,0,5.255429,5.361905,4.051048


In [71]:
data.to_csv('calculated.csv')